In [1]:
import pandas as pd
import numpy as np
import re

# Funções utilizadas

In [3]:
# @title Criando o dataframe
# Inicializando os vetores que serão utilizados para receber os times nas respectivas fases
campeao = []
vice = []
finalistas = []
participantes = []
# Colunas do Dataframe
colunas_df = ['Time', 'CON_SC_1', 'LIG_SC_1']
df_copa = pd.DataFrame(columns=colunas_df)
df_copa

,Time,CON_SC_1,LIG_SC_1


Dicionário de colunas



| Nome | Descrição |
|------|-----------|
| CON_SC_1  | Vencedor da Recopa Sulamericana |
| LIG_SC_1  | Vencedor da Supercopa do Brasil |

In [4]:
df = pd.read_html('https://pt.wikipedia.org/wiki/Recopa_Sul-Americana#Por_equipe')
df = df[4]
df['Títulos'] = df['Títulos'].replace(r'\s*\(.*\)', '', regex=True) # Retirar os parênteses
df['Títulos'] = df['Títulos'].astype(int) # Transformar as colunas em int
df = df[(df['País'] == 'Brasil') & (df['Títulos'] > 0)]
df

,Clube,País,Títulos,Vices
2,São Paulo,Brasil,2,2 (2006 e 2013)
3,Internacional,Brasil,2,1 (2009)
6,Grêmio,Brasil,2,0
8,Cruzeiro,Brasil,1,2 (1992 e 1993)
11,Flamengo,Brasil,1,1 (2023)
12,Palmeiras,Brasil,1,1 (2021)
18,Santos,Brasil,1,0
19,Corinthians,Brasil,1,0
20,Atlético Mineiro,Brasil,1,0
22,Fluminense,Brasil,1,0


In [5]:
df_copa['Time'] = df['Clube']
df_copa['CON_SC_1'] = df['Títulos']
df_copa

,Time,CON_SC_1,LIG_SC_1
2,São Paulo,2,NaN
3,Internacional,2,NaN
6,Grêmio,2,NaN
8,Cruzeiro,1,NaN
11,Flamengo,1,NaN
12,Palmeiras,1,NaN
18,Santos,1,NaN
19,Corinthians,1,NaN
20,Atlético Mineiro,1,NaN
22,Fluminense,1,NaN


In [6]:
df = pd.read_html('https://pt.wikipedia.org/wiki/Supercopa_Rei#Por_clube')
df = df[6]
df.drop(9, inplace=True)
df

,Clube,Títulos,Vices,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Flamengo,"3 (2020, 2021 e 2025)","3 (1991, 2022 e 2023)",NaN,NaN,NaN
1,Palmeiras,1 (2023),2 (2021 e 2024),NaN,NaN,NaN
2,Grêmio,1 (1990),0,NaN,NaN,NaN
3,Corinthians,1 (1991),0,NaN,NaN,NaN
4,Atlético Mineiro,1 (2022),0,NaN,NaN,NaN
5,São Paulo,1 (2024),0,NaN,NaN,NaN
6,Vasco da Gama,0,1 (1990),NaN,NaN,NaN
7,Athletico Paranaense,0,1 (2020),NaN,NaN,NaN
8,Botafogo,0,1 (2025),NaN,NaN,NaN


In [7]:
df = pd.read_html('https://pt.wikipedia.org/wiki/Supercopa_Rei#Por_clube')
df = df[6]
df.drop(9, inplace=True)
df['Títulos'] = df['Títulos'].replace(r'\s*\(.*\)', '', regex=True) # Retirar os parênteses
df['Títulos'] = df['Títulos'].astype(int) # Transformar as colunas em int
df = df[(df['Títulos'] > 0)]
df

,Clube,Títulos,Vices,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Flamengo,3,"3 (1991, 2022 e 2023)",NaN,NaN,NaN
1,Palmeiras,1,2 (2021 e 2024),NaN,NaN,NaN
2,Grêmio,1,0,NaN,NaN,NaN
3,Corinthians,1,0,NaN,NaN,NaN
4,Atlético Mineiro,1,0,NaN,NaN,NaN
5,São Paulo,1,0,NaN,NaN,NaN


In [8]:
df_merged = pd.merge(df_copa, df[['Clube', 'Títulos']], left_on='Time', right_on='Clube', how='left')
# Preenchendo a coluna 'LIG_SC_1' com os valores de 'Títulos'
df_merged['LIG_SC_1'] = df_merged['Títulos']

# Removendo as colunas desnecessárias (como 'Clube' e 'Títulos')
df_merged = df_merged.drop(columns=['Clube', 'Títulos'])
df_merged = df_merged.fillna(0)
df_merged['LIG_SC_1'] = df_merged['LIG_SC_1'].astype(int)
df_copa = df_merged
df_copa

,Time,CON_SC_1,LIG_SC_1
0,São Paulo,2,1
1,Internacional,2,0
2,Grêmio,2,1
3,Cruzeiro,1,0
4,Flamengo,1,3
5,Palmeiras,1,1
6,Santos,1,0
7,Corinthians,1,1
8,Atlético Mineiro,1,1
9,Fluminense,1,0


In [9]:
# @title Função get Table Malformatada
def get_table_unformatted(dataframe):
  # Remover colunas multinivel e colunas que não serão utilizadas
  dataframe = dataframe.drop(dataframe.columns[-2:], axis=1)
  dataframe.columns = dataframe.columns.droplevel(0)
  dataframe.columns = dataframe.columns.get_level_values(0)
  dataframe['Resultado'] = ""
  res = "Campeão"
  indices = []
  for index, row in dataframe.iterrows():
      time = row['Times']
      if 'campeão' in time or 'Eliminados' in time:
        res = time
        indices.append(index)
      else:
        dataframe.at[index, 'Resultado'] = res
  dataframe.drop(indices, inplace=True)
  return dataframe

In [10]:
# @title Função limpar nomes
# Usando regex para limpar o '(x)' no final do nome do time
def clear_text(text):
    text = pd.Series(text).replace(r'\[.*?\]', '', regex=True).replace(r'\(.*?\)', '', regex=True)
    # Remove espaços em branco no início e no final
    return text.str.strip()

In [11]:
# @title Função get Bracket
# Esta função pega o chaveamento da copa na Wikipedia
def get_bracket(final, bracket):
  top_sf = []
  top_qf = []
  top_of = []
  top_f1 = []
  bracket = bracket.loc[2:, :]
  participantes = final
  print(f'Finalistas: {final}')

  # A partir do final do Chaveamento anda as colunas até a fase inicial para recuperar os times
  for i in range(len(bracket.columns)-1, 0, -1):

      # Pegar os times sem repetição na coluna desejada
      times = bracket.loc[:, i].dropna().unique()
      # Retira os possíveis () e []
      times = pd.Series(times).replace(r'\[.*?\]', '', regex=True).replace(r'\(.*?\)', '', regex=True)
      # Define o tipo como String
      times = times.astype(str)
      # Garante que o conteúdo não contém apenas números
      times = times[~times.str.match(r'^[0-9\s]+$')]
      times = times.to_list()
      # Limpa o conteúdo
      times = [team.strip() for team in times if isinstance(team, str) and re.match(r'^(?!\d+$)[A-Za-zÀ-ÿ0-9\s\-]+$', team)]

      # Se a quantidade de times retornada for igual a 4, quer dizer que a coluna é a semifinal
      if (len(times) == 4) & (len(top_sf) == 0):
          # Recupera os times que ainda não estão na lista 'participantes'
          top_sf = [team for team in times if team not in participantes]
          print(f'Semis:  {top_sf}')
          # Adiciona os times da semifinal à lista 'participantes'
          participantes = set(participantes).union(top_sf)

      elif (len(times) == 8) & (len(top_qf) == 0): # 8 times = coluna Quartas de final
          top_qf = [team for team in times if team not in participantes]
          print(f'Quartas: {top_qf}')
          participantes = set(participantes).union(top_qf)

      elif (len(times) == 16) & (len(top_of) == 0): # 16 times = coluna Oitavas de final
          top_of = [team for team in times if team not in participantes]
          print(f'Oitavas: {top_of}')
          participantes = set(participantes).union(top_of)

      elif (len(times) > 16) & (len(times) <= 32) & (len(top_f1) == 0): # Entre 16 e 32 times = coluna -1F
          top_f1 = [team for team in times if team not in participantes]
          print(f'1ª Fase: {top_f1}')
          participantes = set(participantes).union(top_f1)

  return participantes, top_sf, top_qf, top_of, top_f1

In [12]:
# @title Função get Table
def get_table(table, ano):
  tabela = table.rename(columns={table.columns[0]: 'Pos', table.columns[1]: 'Time'})
  #dftemp['Pos'] = dftemp['Pos'].astype(int)
  campeao = []
  vice = []
  top_sf = []
  top_qf = []
  top_of = []
  for i, row in tabela.iterrows():
    pos = float(row["Pos"])
    time = clear_text(row["Time"]).to_list()
    #last_column_value = list(getattr(row, '_asdict')().values())[-1]
    last_column_value = row.iloc[table.shape[1]-1]
    # 88-89: 4 rebaixados
    # 90-91, 94-95: 2 rebaixados
    # 93: 8 rebaixados
    if pos == 1:
      campeao = time
    elif pos == 2:
      vice = time
    elif (pos <= 4):
      top_sf = top_sf + time
    elif (pos <= 8):
      top_qf = top_qf + time
    elif (pos <= 12):
      top_12 = top_12 + time
    elif 'rebaixa' in str(last_column_value).lower():  # transforma o valor em string e faz a comparação
      reb = reb + time
    elif ((ano >= 1988) & (ano != 1992) & (ano != 1996) &
      (ano != 1999) & (ano != 2000)): # Anos com rebaixamento
      if (ano <= 1989) & (pos > len(tabela)-4): # 88-89
        reb = reb + time
      elif (ano <= 1995) & (pos > len(tabela)-2): # 90-91, 94-95
        reb = reb + time
      elif (ano == 2003) & (pos > len(tabela)-2): # 2003
        print(ano)
        reb = reb + time
      elif (ano >= 1996) & (ano != 2003) & (pos > len(tabela)-4): # 96-98, 01-02, 03-hoje
        reb = reb + time
      else:
        top_of = top_of + time
    elif (len(reb) > 0):
        top_of = top_of + time
    else: # Ano sem rebaixamento
      if (pos <= 16):
        top_16 = top_16 + time
      elif (pos <= 24):
        top_24 = top_24 + time
      elif (pos <= 32):
        top_32 = top_32 + time
      elif (pos <= 64):
        top_64 = top_64 + time
      elif (pos <= 100):
        top_100 = top_100 + time
      #top_of = top_of + time
      qtd_ult = round(len(tabela)*0.2)
      if (pos > len(table)-qtd_ult):
        ultimos = ultimos + time

  print(f'Participantes: {len(tabela)}')
  print(f'Campeão: {campeao}')
  print(f'Vice: {vice}')
  print(f'3º e 4º: {top_sf}')
  print(f'5º ao 8º: {top_qf}')
  print(f'9º ao 12º: {top_12}')
  if (len(reb) > 0):
    print(f'Não Rebaixados: {top_of}')
    print(f'Rebaixados: {reb}')
  elif (len(table) > 12):
    print(f'Top 16: {top_16}')
    if (len(top_24) > 0):
      print(f'Top 24: {top_24}')
    if (len(top_32) > 0):
      print(f'Top 32: {top_32}')
    if (len(top_64) > 0):
      print(f'Top 64: {top_64}')
    if (len(top_100) > 0):
      print(f'Top 100: {top_100}')
    print(f'Últimos 20%: {ultimos}')
  participantes = campeao + vice + top_sf + top_qf + top_12 + top_of + reb + top_16 + top_24 + top_32 + top_64 + top_100

  return participantes, top_of, reb, ultimos

In [13]:
# @title Função preencher tabela
def preencher_tabela(times_ed, df):
  times_tabela = df['Time'].to_list()
  # Itera sobre os participantes da edição
  for pos, team in enumerate(times_ed):
      pos += 1
      # Insere o time caso seja a primeira participação
      if team not in times_tabela:
          print(f'Novo participante: {team}')
          df.loc[len(df)] = [team] + [0] * (len(df.columns) - 1)
          # df.loc[len(df)]

      if pos == 1:
          df.loc[df['Time'] == team, 'LIG_A_1'] += 1
      elif pos == 2:
          df.loc[df['Time'] == team, 'LIG_A_2'] += 1
      elif (pos <= 4):
          df.loc[df['Time'] == team, 'LIG_A_4'] += 1
      elif (pos <= 8):
          df.loc[df['Time'] == team, 'LIG_A_8'] += 1
      elif (pos <= 12):
          df.loc[df['Time'] == team, 'LIG_A_12'] += 1
      elif team in reb:
          df.loc[df['Time'] == team, 'LIG_A_R'] += 1
      elif team in top_of:
          df.loc[df['Time'] == team, 'LIG_A_NR'] += 1
      elif (pos <= 16):
          df.loc[df['Time'] == team, 'LIG_A_16'] += 1
      elif (pos <= 24):
          df.loc[df['Time'] == team, 'LIG_A_24'] += 1
      elif (pos <= 32):
          df.loc[df['Time'] == team, 'LIG_A_32'] += 1
      elif (pos <= 64):
          df.loc[df['Time'] == team, 'LIG_A_64'] += 1
      elif (pos <= 100):
          df.loc[df['Time'] == team, 'LIG_A_100'] += 1

      if team in ultimos:
          df.loc[df['Time'] == team, 'LIG_A_ULT'] += 1

      # Adiciona a participação do time na edição
      df.loc[df['Time'] == team, 'LIG_A_P'] += 1

  return df

# Pontuação

| Pct %   | Resultado | Recopa SulAm | Supercopa Rei |
|-|-|-|-|
|         |           | 5% da média Libert+Sula | 5% da Série A |
|
| 100.00% | Campeão   | 235 | 120
|   0.00% | Vice      | 0 | 0

In [14]:
# Pontuação em relação apenas ao Campeonato Brasileiro
df_rank = df_copa.copy()
df_rank['PTS'] = (df_rank['CON_SC_1']*235 +
                  df_rank['LIG_SC_1']*120)

In [15]:
df_rank = df_rank.sort_values(by=['PTS', 'Time'], ascending=[False, True]).reset_index(drop=True)
df_rank.index += 1

# Ranking Geral

In [16]:
df_rank

,Time,CON_SC_1,LIG_SC_1,PTS
1,Flamengo,1,3,595
2,Grêmio,2,1,590
3,São Paulo,2,1,590
4,Internacional,2,0,470
5,Atlético Mineiro,1,1,355
6,Corinthians,1,1,355
7,Palmeiras,1,1,355
8,Cruzeiro,1,0,235
9,Fluminense,1,0,235
10,Santos,1,0,235


In [17]:
# Exportar para Excel
df_rank.to_excel('Supercopas Rank.xlsx')